In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from scipy.spatial import cKDTree

In [2]:
from methods import link_df_to_geojson
from methods import point_df_to_geojson
from methods import identify_dead_end_nodes

In [3]:
data_interim_dir = "../../data/interim/"

data_external_dir = "../../data/external/"

# Read network

In [501]:
shape_gdf = gpd.read_file(data_interim_dir + "step3_join_shst_extraction_with_osm/" 
                          + "shape.geojson")

In [502]:
node_file = data_interim_dir + "step3_join_shst_extraction_with_osm/" + "node.geojson"
node_gdf = gpd.read_file(node_file)

In [503]:
link_file = data_interim_dir + "step4_conflate_with_tomtom/" + "link.feather"

link_df = pd.read_feather(link_file)

In [504]:
shape_gdf.crs

{'init': 'epsg:4326'}

In [505]:
link_df[link_df.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,service,shstGeometryId,shstReferenceId,toIntersectionId,tunnel,u,v,walk_access,wayId,width
348788,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,nan,b5a13d71332300baf271a3dde50dd06d,6233a20ff0d9dc29be597a06ffd19727,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan


# Join county name to shapes and nodes

In [506]:
county_file = data_external_dir + "county_boundaries/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"

county_gdf = gpd.read_file(county_file)

county_gdf = county_gdf.to_crs(shape_gdf.crs)

In [507]:
node_county_gdf = gpd.sjoin(node_gdf, county_gdf, how = "left", op = "intersects")

In [508]:
node_gdf.shape

(661159, 6)

In [509]:
node_county_gdf.shape

(661160, 16)

In [510]:
# use nearest match for nodes that did not get county match (e.g. in the Bay)

node_county_matched_gdf = node_county_gdf[node_county_gdf.NAME.notnull()].copy()
node_county_unmatched_gdf = node_county_gdf[node_county_gdf.NAME.isnull()].copy()

node_county_matched_gdf = node_county_matched_gdf.to_crs(epsg = 26915)
node_county_matched_gdf['X'] = node_county_matched_gdf.geometry.map(lambda g:g.x)
node_county_matched_gdf['Y'] = node_county_matched_gdf.geometry.map(lambda g:g.y)

node_matched_inventory_ref = node_county_matched_gdf[['X', 'Y']].values
node_matched_tree = cKDTree(node_matched_inventory_ref)

node_county_unmatched_gdf = node_county_unmatched_gdf.to_crs({'init' : 'epsg:26915'})
node_county_unmatched_gdf['X'] = node_county_unmatched_gdf['geometry'].apply(lambda p: p.x)
node_county_unmatched_gdf['Y'] = node_county_unmatched_gdf['geometry'].apply(lambda p: p.y)

node_county_rematch_gdf = pd.DataFrame()

for i in range(len(node_county_unmatched_gdf)):
    point = node_county_unmatched_gdf.iloc[i][['X', 'Y']].values
    dd, ii = node_matched_tree.query(point, k = 1)
    add_snap_gdf = gpd.GeoDataFrame(node_county_matched_gdf.iloc[ii][["NAME"]]).transpose().reset_index(drop = True)
    
    add_snap_gdf['shst_node_id'] = node_county_unmatched_gdf.iloc[i]['shst_node_id']
    
    if i == 0:
        node_county_rematch_gdf = add_snap_gdf.copy()
    else:
        node_county_rematch_gdf = node_county_rematch_gdf.append(add_snap_gdf, ignore_index=True, sort=False)

In [511]:
node_county_rematch_dict = dict(zip(node_county_rematch_gdf.shst_node_id, node_county_rematch_gdf.NAME))

In [512]:
node_county_gdf["NAME"] = node_county_gdf["NAME"].fillna(node_county_gdf.shst_node_id.map(node_county_rematch_dict))

In [513]:
node_county_gdf.NAME.value_counts()

Santa Clara      193106
Alameda          126021
Contra Costa      97374
San Mateo         56430
Sonoma            56166
Solano            47305
San Francisco     27656
Marin             26543
Napa              13838
Santa Cruz         5232
Yolo               4238
Lake               2890
San Joaquin        2120
San Benito         1095
Sacramento          383
Mendocino           324
Stanislaus          268
Monterey            124
Merced               47
Name: NAME, dtype: int64

In [514]:
shape_centroid_gdf = shape_gdf.copy()
shape_centroid_gdf["geometry"] = shape_centroid_gdf["geometry"].centroid

shape_centroid_gdf = gpd.sjoin(shape_centroid_gdf, county_gdf, how = "left", op = "intersects")

shape_county_gdf = pd.merge(
    shape_gdf,
    shape_centroid_gdf[["id", "NAME"]],
    how = "left",
    on = "id"
)

In [515]:
shape_gdf.shape

(908267, 6)

In [516]:
shape_county_gdf.shape

(908267, 7)

In [517]:
shape_county_gdf.NAME.value_counts().sum()

907992

In [518]:
shape_county_gdf[shape_county_gdf.NAME.isnull()]

,id,fromIntersectionId,toIntersectionId,forwardReferenceId,backReferenceId,geometry,NAME
115085,275584284bd772edbbb2dbf9fbb7bfde,e076bbf31b048f22833c391c7ad46bb3,d1915df70b5e646e70c8242ff06d5610,37cf6e02dae25f0ad166881851e79d17,b5889bfe79d125f681bc92f66a6dacfe,"LINESTRING (-122.1286524 37.4989011, -122.1283...",NaN
118263,e486664e0612f56f5ebaf275bd0b7c1b,aa3850bdf7a2925617183d97994f656e,3c98e5b6fb8993c575d67f179944f3d4,0714bd4d50ff52fb1ce5cb716f82d4eb,2a5c372be05b14fc71630c76803cc2d8,"LINESTRING (-122.1214496 37.4764155, -122.1212...",NaN
125376,0ec61d53f6c69ef515ba4009ffede64c,9ec988e879689deddc94e4388e6c5e41,dcbccd5feb582a817a3caedcb590640a,20871cef2e397727dffbfceb3d0bd360,59b0a677a3485914314c42c2c33b2710,"LINESTRING (-122.1529321 37.6418294, -122.1531...",NaN
126053,a4993fc2935164038331907ae2c0ef0f,72f5477ed221e106cad0a2bb449763d9,18001131b3cbb61bffbd46f232f9e116,0a20680ad7283456f4e795179413f9ae,,"LINESTRING (-122.1544879 37.6170117, -122.1536...",NaN
126055,42a0b071676831f8d245e694736f9324,72f5477ed221e106cad0a2bb449763d9,12c427672a8791d94bfcaf5378f8c34f,f7c7b1a57c9076b9c04818964378c0df,,"LINESTRING (-122.1544879 37.6170117, -122.1532...",NaN
126057,87fd1ba651b320516da2d253068922e7,dc47f9816efab67bcc3a6b8e3cb3068b,06e88fc49da5dcab64b5b5a6bd927775,74257e0ff098cc9c2b96e04835359062,,"LINESTRING (-122.1491637 37.6188537, -122.1496...",NaN
207079,cd08b2df696f8b1c28d5d5b0fe5b1b34,90e176fc9bb2c3837689b604acf81568,c8c50e5747d5e343caf6a85031b3993e,6e437e8872936b96a3deda5c42e6ffb3,2446660670456b561a866b41f14ebca2,"LINESTRING (-123.4361553 38.6912158, -123.4363...",NaN
207082,3f985789601fb7cb10ec68ab8582caf2,29f667a7c24bfb268ad1f65e005abfe4,426a523201d2d6db3c7767faba554155,63cb84e26634484a0d231bd1ca47a7d8,ef1b950b3ba3a105f3388cd44e20da00,"LINESTRING (-123.4392507 38.6970098, -123.4389...",NaN
208052,87112cba76db310cf77679e99672c83d,fa8b9299507eb4a25ddcca933134b2dd,b38a520e056d0e4b504b7edb3e9167ff,e45b6d9bee468411ae48e7e8443671df,663b405bfe82c29fb979b7882a2501f1,"LINESTRING (-123.3329598 38.5662247, -123.3329...",NaN
208053,c2d3447dfbadd15c5f3667db273b94f3,1481f6ac41cfaa7b0da87e341c72b7c2,fa8b9299507eb4a25ddcca933134b2dd,5ea536e33e7cc38536c19277cb409e23,d1180a6388809609873b8f96d2f68cff,"LINESTRING (-123.3318745 38.56634, -123.331805...",NaN


In [519]:
# use nearest for links that did not get county match
node_county_matched_gdf = node_county_gdf[node_county_gdf.NAME.notnull()].copy()

node_county_matched_gdf = node_county_matched_gdf.to_crs(epsg = 26915)
node_county_matched_gdf['X'] = node_county_matched_gdf.geometry.map(lambda g:g.x)
node_county_matched_gdf['Y'] = node_county_matched_gdf.geometry.map(lambda g:g.y)

node_matched_inventory_ref = node_county_matched_gdf[['X', 'Y']].values
node_matched_tree = cKDTree(node_matched_inventory_ref)

shape_county_unmatched_gdf = shape_county_gdf[shape_county_gdf.NAME.isnull()].copy()

shape_county_unmatched_gdf = shape_county_unmatched_gdf.to_crs({'init' : 'epsg:26915'})
shape_county_unmatched_gdf["geometry"] = shape_county_unmatched_gdf["geometry"].centroid
shape_county_unmatched_gdf['X'] = shape_county_unmatched_gdf['geometry'].apply(lambda p: p.x)
shape_county_unmatched_gdf['Y'] = shape_county_unmatched_gdf['geometry'].apply(lambda p: p.y)

shape_county_rematch_gdf = pd.DataFrame()

for i in range(len(shape_county_unmatched_gdf)):
    point = shape_county_unmatched_gdf.iloc[i][['X', 'Y']].values
    dd, ii = node_matched_tree.query(point, k = 1)
    add_snap_gdf = gpd.GeoDataFrame(node_county_matched_gdf.iloc[ii][["NAME"]]).transpose().reset_index(drop = True)
    
    add_snap_gdf['id'] = shape_county_unmatched_gdf.iloc[i]['id']
    
    if i == 0:
        shape_county_rematch_gdf = add_snap_gdf.copy()
    else:
        shape_county_rematch_gdf = shape_county_rematch_gdf.append(add_snap_gdf, ignore_index=True, sort=False)

In [520]:
shape_county_rematch_dict = dict(zip(shape_county_rematch_gdf.id, shape_county_rematch_gdf.NAME))

shape_county_gdf["NAME"] = shape_county_gdf["NAME"].fillna(shape_county_gdf.id.map(shape_county_rematch_dict))

# Remove duplicate shape/node in county match, e.g. geometry on the boundary

In [521]:
node_county_gdf.drop_duplicates(subset = ["shst_node_id"], inplace = True)

shape_county_gdf.drop_duplicates(subset = ["id"], inplace = True)

In [522]:
node_gdf.shape

(661159, 6)

In [523]:
node_county_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 661159 entries, 0 to 661158
Data columns (total 16 columns):
osm_node_id     661159 non-null int64
shst_node_id    661159 non-null object
drive_access    661159 non-null int64
walk_access     661159 non-null int64
bike_access     661159 non-null int64
geometry        661159 non-null object
index_right     660885 non-null float64
STATEFP         660885 non-null object
COUNTYFP        660885 non-null object
COUNTYNS        660885 non-null object
AFFGEOID        660885 non-null object
GEOID           660885 non-null object
NAME            661159 non-null object
LSAD            660885 non-null object
ALAND           660885 non-null float64
AWATER          660885 non-null float64
dtypes: float64(3), int64(4), object(9)
memory usage: 85.8+ MB


In [524]:
shape_gdf.shape

(908267, 6)

In [525]:
shape_county_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 908267 entries, 0 to 908266
Data columns (total 7 columns):
id                    908267 non-null object
fromIntersectionId    908267 non-null object
toIntersectionId      908267 non-null object
forwardReferenceId    908267 non-null object
backReferenceId       908267 non-null object
geometry              908267 non-null object
NAME                  908267 non-null object
dtypes: object(7)
memory usage: 55.4+ MB


In [526]:
shape_county_gdf[shape_county_gdf.id == "b5a13d71332300baf271a3dde50dd06d"]

,id,fromIntersectionId,toIntersectionId,forwardReferenceId,backReferenceId,geometry,NAME
482767,b5a13d71332300baf271a3dde50dd06d,f407efc156163c556b516200a966187e,7da9cd62c681f3425ba9b2e28d702fc9,6233a20ff0d9dc29be597a06ffd19727,239f1586cff1f2116ad5e26df9375927,"LINESTRING (-122.2339452 37.4206079, -122.2341...",San Mateo


# Remove links and nodes outside of the 9 counties

In [527]:
MPO_county_list = ['San Francisco', 'Santa Clara', 'Sonoma', 'Marin', 'San Mateo',
       'Contra Costa', 'Solano', 'Napa', 'Alameda']

In [528]:
node_gdf = pd.merge(
    node_gdf, 
    node_county_gdf[["shst_node_id", "NAME"]].rename(columns = {"NAME": "county"}), 
    how = "left", 
    on = "shst_node_id") 

In [529]:
node_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 661159 entries, 0 to 661158
Data columns (total 7 columns):
osm_node_id     661159 non-null int64
shst_node_id    661159 non-null object
drive_access    661159 non-null int64
walk_access     661159 non-null int64
bike_access     661159 non-null int64
geometry        661159 non-null object
county          661159 non-null object
dtypes: int64(4), object(3)
memory usage: 40.4+ MB


In [530]:
shape_MPO_gdf = shape_county_gdf[shape_county_gdf.NAME.isin(MPO_county_list)].copy()

In [531]:
shape_MPO_gdf.columns

Index(['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId',
       'backReferenceId', 'geometry', 'NAME'],
      dtype='object')

In [532]:
shape_MPO_gdf.shape

(888188, 7)

In [533]:
shape_MPO_gdf[shape_MPO_gdf.id == "b5a13d71332300baf271a3dde50dd06d"]

,id,fromIntersectionId,toIntersectionId,forwardReferenceId,backReferenceId,geometry,NAME
482767,b5a13d71332300baf271a3dde50dd06d,f407efc156163c556b516200a966187e,7da9cd62c681f3425ba9b2e28d702fc9,6233a20ff0d9dc29be597a06ffd19727,239f1586cff1f2116ad5e26df9375927,"LINESTRING (-122.2339452 37.4206079, -122.2341...",San Mateo


In [534]:
link_df[link_df.u == 5372055804][["roadway", "v", "drive_access", "walk_access", "bike_access"]]

,roadway,v,drive_access,walk_access,bike_access
142409,footway,5000827329,0,1,0
548904,cycleway,890045129,0,1,1
1221973,footway,5000826420,0,1,0


In [535]:
link_df[link_df.v == 5372055804][["roadway", "u", "drive_access", "walk_access", "bike_access"]]

,roadway,u,drive_access,walk_access,bike_access
338507,footway,5000826420,0,1,0
903556,service,890045140,1,1,1
926879,footway,5000827329,0,1,0


In [536]:
link_MPO_df = link_df[link_df.shstGeometryId.isin(shape_MPO_gdf.id.tolist())].copy()

node_MPO_gdf = node_gdf[node_gdf.shst_node_id.isin(link_MPO_df.fromIntersectionId.tolist() + 
                                                   link_MPO_df.toIntersectionId.tolist())].copy()

In [537]:
# nodes that are outside of MPO but used by MPO links, need to give them the internal county names for node numbering

node_MPO_rename_county_gdf = node_MPO_gdf[~node_MPO_gdf.county.isin(MPO_county_list)].copy()

node_link_county_names_df = pd.concat(
    [
        shape_MPO_gdf.groupby(["fromIntersectionId", "NAME"])["id"].count().reset_index().rename(
            columns = {"fromIntersectionId" : "shst_node_id", "NAME" : "county"}),
        shape_MPO_gdf.groupby(["toIntersectionId", "NAME"])["id"].count().reset_index().rename(
            columns = {"toIntersectionId" : "shst_node_id", "NAME" : "county"})
    ],
    sort = False,
    ignore_index = True
)

node_MPO_rename_county_gdf = pd.merge(
    node_MPO_rename_county_gdf.drop(["county"], axis = 1),
    node_link_county_names_df[["shst_node_id", "county"]],
    how = "left",
    on = "shst_node_id"
)

node_MPO_rename_county_gdf.drop_duplicates(subset = ["osm_node_id", "shst_node_id"], inplace = True)

In [538]:
node_MPO_rename_county_gdf.head(3)

,osm_node_id,shst_node_id,drive_access,walk_access,bike_access,geometry,county
0,90535719,9ac4414af3095d8b2fd034dda4ac6330,1,1,1,POINT (-121.7016325 38.1050639),Solano
1,86017483,1af41a838d805cc4ef50c28aa508f6da,1,1,1,POINT (-123.231362 38.81535),Sonoma
2,5040098253,0be862b72977b1f6e8398628c22be835,1,1,1,POINT (-121.5568837 37.7856817),Alameda


In [539]:
node_MPO_gdf.columns

Index(['osm_node_id', 'shst_node_id', 'drive_access', 'walk_access',
       'bike_access', 'geometry', 'county'],
      dtype='object')

In [540]:
node_MPO_rename_county_gdf.columns

Index(['osm_node_id', 'shst_node_id', 'drive_access', 'walk_access',
       'bike_access', 'geometry', 'county'],
      dtype='object')

In [541]:
node_MPO_gdf = pd.concat(
    [
        node_MPO_gdf[node_MPO_gdf.county.isin(MPO_county_list)],
        node_MPO_rename_county_gdf
    ],
    sort = False,
    ignore_index = True
)

In [542]:
link_MPO_gdf = pd.merge(
    link_MPO_df,
    shape_MPO_gdf[["id", "NAME", "geometry"]].rename(columns = {"NAME" : "county"}),
    how = "left", 
    on = "id"
)

link_MPO_gdf = gpd.GeoDataFrame(link_MPO_gdf, geometry = link_MPO_gdf["geometry"],
                            crs={"init" : "epsg:4326"})

In [543]:
link_MPO_gdf[link_MPO_gdf.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,shstReferenceId,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry
341082,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,6233a20ff0d9dc29be597a06ffd19727,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341..."


In [544]:
link_MPO_gdf.county.value_counts()

Santa Clara      524820
Alameda          321922
Contra Costa     243348
San Mateo        144181
Sonoma           138807
Solano           119966
San Francisco     74441
Marin             64637
Napa              34622
Name: county, dtype: int64

In [545]:
print(link_MPO_gdf.drive_access.value_counts())
print(link_MPO_gdf.walk_access.value_counts())
print(link_MPO_gdf.bike_access.value_counts())

1    1313716
0     353028
Name: drive_access, dtype: int64
1    1663966
0       2778
Name: walk_access, dtype: int64
1    1413044
0     253700
Name: bike_access, dtype: int64


In [546]:
node_MPO_gdf.county.value_counts()

Santa Clara      193195
Alameda          126035
Contra Costa      97375
San Mateo         56453
Sonoma            56183
Solano            47336
San Francisco     27656
Marin             26543
Napa              13841
Name: county, dtype: int64

In [547]:
print(node_MPO_gdf.drive_access.value_counts())
print(node_MPO_gdf.walk_access.value_counts())
print(node_MPO_gdf.bike_access.value_counts())

1    545584
0     99033
Name: drive_access, dtype: int64
1    644489
0       128
Name: walk_access, dtype: int64
1    581949
0     62668
Name: bike_access, dtype: int64


In [548]:
len(set(link_MPO_gdf.u.tolist() + link_MPO_gdf.v.tolist()))

644617

In [549]:
node_MPO_gdf.shape

(644617, 7)

# Add length

In [550]:
# add length in meters

geom_length = link_MPO_gdf[['geometry']].copy()
geom_length = geom_length.to_crs(epsg = 26915)
geom_length["length"] = geom_length.length

link_MPO_gdf["length"] = geom_length["length"]

# drop circular links (u == v)

In [551]:
circular_link_gdf = link_MPO_gdf[link_MPO_gdf.u == link_MPO_gdf.v].copy()
circular_link_gdf.shape

(5560, 33)

In [552]:
circular_link_gdf.id.nunique()

4921

In [553]:
link_MPO_gdf.shape

(1666744, 33)

In [554]:
link_MPO_gdf.shstReferenceId.nunique()

1666744

In [555]:
link_MPO_gdf[link_MPO_gdf.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
341082,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341...",314.736276


In [556]:
link_MPO_gdf = link_MPO_gdf[~ link_MPO_gdf.shstReferenceId.isin(circular_link_gdf.shstReferenceId.tolist())]

In [557]:
link_MPO_gdf[link_MPO_gdf.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
341082,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341...",314.736276


In [558]:
link_MPO_gdf.shape

(1661184, 33)

In [559]:
link_MPO_gdf.shstReferenceId.nunique()

1661184

In [560]:
link_MPO_gdf.id.nunique()

883267

In [561]:
shape_MPO_gdf = shape_MPO_gdf[shape_MPO_gdf.id.isin(link_MPO_gdf.id)]

In [562]:
shape_MPO_gdf.id.nunique()

883267

In [563]:
node_MPO_gdf = node_MPO_gdf[(node_MPO_gdf.osm_node_id.isin(link_MPO_gdf.u.tolist())) | 
                            (node_MPO_gdf.osm_node_id.isin(link_MPO_gdf.v.tolist()))]

In [564]:
node_MPO_gdf.shape

(643811, 7)

# Flag drive dead end

In [565]:
non_dead_end_link_handle_df = link_MPO_gdf[(link_MPO_gdf.drive_access == 1)][["u", "v"]]

dead_end_node_list = identify_dead_end_nodes(non_dead_end_link_handle_df)

cumulative_dead_end_node_list = []

while len(dead_end_node_list) > 0:
    cumulative_dead_end_node_list = cumulative_dead_end_node_list + dead_end_node_list
    
    non_dead_end_link_handle_df = non_dead_end_link_handle_df[~(non_dead_end_link_handle_df.u.isin(dead_end_node_list)) & 
                                            ~(non_dead_end_link_handle_df.v.isin(dead_end_node_list))].copy()
    
    dead_end_node_list = identify_dead_end_nodes(non_dead_end_link_handle_df)

In [566]:
len(cumulative_dead_end_node_list)

142963

In [567]:
5372055804 in cumulative_dead_end_node_list

True

In [568]:
link_df[link_df.u == 5372055804].v

142409     5000827329
548904      890045129
1221973    5000826420
Name: v, dtype: int64

In [569]:
non_dead_end_link_handle_df

,u,v
0,2401244716,2401244712
1,57839068,57869731
2,1024388950,110424978
3,65561433,1325928459
4,4545575571,4545575563
5,3377850399,4313871331
6,4305402275,4305402282
8,4925258551,4332522355
9,65308539,65318508
10,4932240373,4616479310


In [570]:
link_MPO_gdf.drive_access.value_counts()

1    1309170
0     352014
Name: drive_access, dtype: int64

In [571]:
node_MPO_gdf.drive_access.value_counts()

1    545259
0     98552
Name: drive_access, dtype: int64

In [572]:
# update node and link drive access
# if u/v in dead end node list, then drive access = 0
# if osm_node_id in dead end node list, then drive access = 0

link_MPO_gdf['drive_access'] = np.where(((link_MPO_gdf.u.isin(cumulative_dead_end_node_list)) | 
                                               (link_MPO_gdf.v.isin(cumulative_dead_end_node_list))) &
                                        ~(link_MPO_gdf.roadway.isin(['primary', 'secondary', 'motorway', 'primary_link',
                                               'motorway_link', 'trunk_link', 'trunk', 'secondary_link',
                                               'tertiary_link'])),
                                               0,
                                               link_MPO_gdf.drive_access)

In [573]:
# update network type variable for node

A_B_df = pd.concat([link_MPO_gdf[["u", "fromIntersectionId", "drive_access", "walk_access", "bike_access"]].rename(
                            columns = {"u":"osm_node_id", "fromIntersectionId" : "shst_node_id"}),
                  link_MPO_gdf[["v", "toIntersectionId", "drive_access", "walk_access", "bike_access"]].rename(
                            columns = {"v":"osm_node_id", "toIntersectionId" : "shst_node_id"})],
                  sort = False,
                  ignore_index = True)

A_B_df.drop_duplicates(inplace = True)

A_B_df = A_B_df.groupby(["osm_node_id", "shst_node_id"]).max().reset_index()

node_MPO_gdf = pd.merge(node_MPO_gdf.drop(["drive_access", "walk_access", "bike_access"], axis = 1),
                      A_B_df,
                      how = "left",
                      on = ["osm_node_id", "shst_node_id"])

In [574]:
A_B_df.shape

(643811, 5)

In [575]:
link_MPO_gdf.drive_access.value_counts()

1    1019540
0     641644
Name: drive_access, dtype: int64

In [576]:
node_MPO_gdf.shape

(643811, 7)

In [577]:
node_MPO_gdf.drive_access.value_counts()

1    402361
0    241450
Name: drive_access, dtype: int64

In [578]:
link_MPO_gdf[(link_MPO_gdf.drive_access == 0) & (link_MPO_gdf.walk_access == 0) & (link_MPO_gdf.bike_access == 0)].roadway

Series([], Name: roadway, dtype: object)

In [579]:
link_MPO_gdf[link_MPO_gdf.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
341082,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341...",314.736276


In [580]:
link_df.roadway.unique()

array(['service', 'residential', 'tertiary', 'primary', 'cycleway',
       'footway', 'secondary', 'motorway', 'primary_link',
       'motorway_link', 'trunk_link', 'trunk', 'secondary_link',
       'tertiary_link'], dtype=object)

In [581]:
link_MPO_gdf[(link_MPO_gdf.drive_access == 0)].roadway.value_counts()

footway        250682
service        156349
residential    132552
cycleway       101332
tertiary          729
Name: roadway, dtype: int64

In [582]:
node_MPO_gdf[node_MPO_gdf.shst_node_id.isin(node_MPO_rename_county_gdf.shst_node_id.tolist())]

,osm_node_id,shst_node_id,geometry,county,drive_access,walk_access,bike_access
643584,90535719,9ac4414af3095d8b2fd034dda4ac6330,POINT (-121.7016325 38.1050639),Solano,1,1,1
643585,86017483,1af41a838d805cc4ef50c28aa508f6da,POINT (-123.231362 38.81535),Sonoma,0,1,1
643586,5040098253,0be862b72977b1f6e8398628c22be835,POINT (-121.5568837 37.7856817),Alameda,0,1,1
643587,95722215,5d60a0cb1999ab51c0a043b8660a310a,POINT (-121.695261 38.514734),Solano,0,1,1
643588,624524794,5ce7982c8f26842d124adb37a07fbd2c,POINT (-121.7568325 37.0491931),Santa Clara,0,1,1
643589,89445717,c3faaf4edfb3666650660c48d8602251,POINT (-121.217535 37.122678),Santa Clara,0,1,1
643590,4003083628,acdea6b75887fafaf03ecd5341df6360,POINT (-121.5373077 37.8429623),Contra Costa,0,1,1
643591,4993578363,57e57e3ba4c52ba8948a03b870ad9aa1,POINT (-121.5564266 37.7588262),Alameda,0,1,1
643592,86452091,8069a166d33ca9002cc0e0d01f98e711,POINT (-123.220412 38.818201),Sonoma,0,1,1
643593,62394174,c0e3f6a6ad024fd6754e19228134509f,POINT (-122.0949798 38.5137149),Solano,1,1,1


In [583]:
link_MPO_gdf[link_MPO_gdf.v == 62394174]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
1252258,"['nan', 'nan']","['nan', 'nan']",1,"['yes', 'nan']",1,"['nan', 'nan']",b6e691d77bc53a92d4f6a62e91301ccc,"['secondary', 'secondary']",3d6d41eef3868e2d4448c5edd8703bc6,"['nan', 'nan']",...,c0e3f6a6ad024fd6754e19228134509f,"['nan', 'nan']",387990215,62394174,1,"[8713932, 25749628]","['nan', 'nan']",Solano,"LINESTRING (-122.0949798 38.5137149, -122.0952...",211.216796


# Drop duplicate links between same AB node pair

In [584]:
non_unique_AB_links_df = link_MPO_gdf.groupby(["u", "v"]).shstReferenceId.count().sort_values().reset_index()
non_unique_AB_links_df = non_unique_AB_links_df[non_unique_AB_links_df.shstReferenceId > 1]

non_unique_AB_links_df = pd.merge(non_unique_AB_links_df[["u", "v"]],
                              link_MPO_gdf[["u", "v", "highway", "roadway", "drive_access", "bike_access", "walk_access", "length",
                                      "wayId", "shstGeometryId", "shstReferenceId", "geometry"]],
                              how = "left",
                              on = ["u", "v"])

In [585]:
non_unique_AB_links_df.roadway.value_counts()

service          34111
residential      10503
footway           5318
cycleway          5021
tertiary           876
secondary          627
primary            266
trunk               20
motorway            10
trunk_link           4
primary_link         3
motorway_link        3
Name: roadway, dtype: int64

In [586]:
link_MPO_gdf[(link_MPO_gdf.u == 4075004794) & (link_MPO_gdf.v == 4075001784)]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
341082,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341...",314.736276
1104417,"['nan', 'nan']","['nan', 'nan']",1,"['nan', 'nan']",0,"['nan', 'nan']",f407efc156163c556b516200a966187e,"['service', 'track']",b6f6f29894314a39e1b7069bd4f53d96,"['nan', 'nan']",...,7da9cd62c681f3425ba9b2e28d702fc9,"['nan', 'nan']",4075004794,4075001784,1,"[405292128, 405292129]","['nan', 'nan']",San Mateo,"LINESTRING (-122.2364996 37.4191189, -122.2365...",753.540618


In [587]:
non_unique_AB_links_df[non_unique_AB_links_df.shstGeometryId == "b5a13d71332300baf271a3dde50dd06d"]

,u,v,highway,roadway,drive_access,bike_access,walk_access,length,wayId,shstGeometryId,shstReferenceId,geometry
24752,4075004794,4075001784,track,cycleway,0,1,1,314.736276,405292130,b5a13d71332300baf271a3dde50dd06d,6233a20ff0d9dc29be597a06ffd19727,"LINESTRING (-122.2339452 37.4206079, -122.2341..."
24755,4075001784,4075004794,track,cycleway,0,1,1,314.736276,405292130,b5a13d71332300baf271a3dde50dd06d,239f1586cff1f2116ad5e26df9375927,"LINESTRING (-122.2339452 37.4206079, -122.2341..."


In [588]:
non_unique_AB_links_df

,u,v,highway,roadway,drive_access,bike_access,walk_access,length,wayId,shstGeometryId,shstReferenceId,geometry
0,5493358061,832260450,service,service,0,1,1,86.998878,571633507,c292f727d9fe1ee89d2f932d8430a26e,7e0b71e0f01f53edb659eb3dac84cd3d,"LINESTRING (-122.0528345 37.6717391, -122.0527..."
1,5493358061,832260450,residential,residential,0,1,1,31.942883,6384706,90af9682dc8d0b560d7e605665b4e4c3,f9a60d2b6947ec00edfa38d4a6258068,"LINESTRING (-122.0531519 37.6718225, -122.0528..."
2,2482062375,2482062433,service,service,1,1,1,70.956734,240472662,a64eaea7a3c5a2d1f13efe765778c5b7,742720d65a97f85124a271f64d86e6fd,"LINESTRING (-122.3057263 37.9939809, -122.3049..."
3,2482062375,2482062433,service,service,1,1,1,96.796573,240472660,87c9a6825ae6ac44b9955b02b46270ec,ebd39faab6f92c45dc0e17b766498c86,"LINESTRING (-122.3049843 37.9939249, -122.3053..."
4,1233081811,1233081567,footway,footway,0,0,1,93.183199,107254877,b5d1cb445985832fb22e4b1e0fa0a407,d5f3dd21781045308b45fda2a0531852,"LINESTRING (-122.1951368 37.7605487, -122.1950..."
5,1233081811,1233081567,footway,footway,0,0,1,82.435365,107254921,23554584c8fe5e0f3c6c21e17ffea0b4,f0a961cc31265126ac3555b668472e79,"LINESTRING (-122.1951368 37.7605487, -122.1952..."
6,4931835557,4931835550,service,service,0,1,1,113.120135,502811696,bcf9a41b094b908b88a4bdeadef7b76a,96838728dbacacd57731a31df56ad185,"LINESTRING (-122.8787877 38.4046133, -122.8787..."
7,4931835557,4931835550,service,service,0,1,1,84.092560,502811696,bd50f5f1b1653b1a4647cb3c5e109d4b,940a390c1ebfc1c5d325a786cf40b9c4,"LINESTRING (-122.8779668 38.4046001, -122.8780..."
8,4930090412,4930090405,service,service,1,1,1,211.750991,502576004,d3a99ae8a5b124abe8a92a8853e46501,56f87be49662475a9c84cb1644d6bec6,"LINESTRING (-121.9517985 38.256374, -121.95173..."
9,4930090412,4930090405,,residential,1,1,1,17.261674,502558445,972a1e9b02614ffa5a32ee9b75dc0fd3,462f81c1c87c6b26efa7f3622b5369e9,"LINESTRING (-121.9519131 38.2562623, -121.9518..."


In [589]:
link_MPO_gdf[link_MPO_gdf.u == link_MPO_gdf.v]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length


In [590]:
non_unique_AB_links_df.shape

(56762, 12)

In [591]:
link_MPO_gdf[link_MPO_gdf.shstGeometryId == "c6aa967814865bf0eec8ebd0e5b27d3c"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
871576,nan,nan,1,nan,0,nan,40b7757ac264ef8fc3deeee11d25514f,residential,c6aa967814865bf0eec8ebd0e5b27d3c,nan,...,4319c448c6e983dc44395113accaae58,nan,55956915,1129773955,1,7699978,nan,Napa,"LINESTRING (-122.2577045 38.1849324, -122.2577...",113.890865
1386565,nan,nan,1,nan,0,nan,4319c448c6e983dc44395113accaae58,residential,c6aa967814865bf0eec8ebd0e5b27d3c,nan,...,40b7757ac264ef8fc3deeee11d25514f,nan,1129773955,55956915,1,7699978,nan,Napa,"LINESTRING (-122.2577045 38.1849324, -122.2577...",113.890865


In [592]:
link_MPO_gdf[link_MPO_gdf.shstGeometryId == "b5a13d71332300baf271a3dde50dd06d"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,toIntersectionId,tunnel,u,v,walk_access,wayId,width,county,geometry,length
341082,nan,nan,1,nan,0,nan,f407efc156163c556b516200a966187e,track,b5a13d71332300baf271a3dde50dd06d,nan,...,7da9cd62c681f3425ba9b2e28d702fc9,nan,4075004794,4075001784,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341...",314.736276
996413,nan,nan,1,nan,0,nan,7da9cd62c681f3425ba9b2e28d702fc9,track,b5a13d71332300baf271a3dde50dd06d,nan,...,f407efc156163c556b516200a966187e,nan,4075001784,4075004794,1,405292130,nan,San Mateo,"LINESTRING (-122.2339452 37.4206079, -122.2341...",314.736276


In [593]:
roadway_hierarchy_df = pd.read_csv("../../data/interim/highway_to_roadway.csv")

In [594]:
roadway_hierarchy_df = roadway_hierarchy_df.drop_duplicates(subset = "roadway")

In [595]:
non_unique_AB_links_df = pd.merge(non_unique_AB_links_df,
                              roadway_hierarchy_df[["roadway", "hierarchy"]],
                              how = "left",
                              on = "roadway")

In [596]:
non_unique_AB_links_df.shape

(56762, 13)

In [597]:
non_unique_AB_links_df[(non_unique_AB_links_df.u == 4075004794) & 
                       (non_unique_AB_links_df.v == 4075001784)]

,u,v,highway,roadway,drive_access,bike_access,walk_access,length,wayId,shstGeometryId,shstReferenceId,geometry,hierarchy
24752,4075004794,4075001784,track,cycleway,0,1,1,314.736276,405292130,b5a13d71332300baf271a3dde50dd06d,6233a20ff0d9dc29be597a06ffd19727,"LINESTRING (-122.2339452 37.4206079, -122.2341...",13
24753,4075004794,4075001784,"['service', 'track']",service,0,1,1,753.540618,"[405292128, 405292129]",b6f6f29894314a39e1b7069bd4f53d96,4715d32ebbf18e592af26b06ce167ac0,"LINESTRING (-122.2364996 37.4191189, -122.2365...",12


In [598]:
# sort on hierarchy (ascending), drive_access(descending), bike_access(descending), walk_access(descending), length(ascending)

non_unique_AB_links_sorted_df =  non_unique_AB_links_df.sort_values(
    by = ["hierarchy", "drive_access", "bike_access", "walk_access", "length"],
    ascending = [True, False, False, False, True])

unique_AB_links_df = non_unique_AB_links_sorted_df.drop_duplicates(subset = ["u", "v"], keep = "first")

In [599]:
unique_AB_links_df[(unique_AB_links_df.u == 4075004794) & 
                       (unique_AB_links_df.v == 4075001784)]

,u,v,highway,roadway,drive_access,bike_access,walk_access,length,wayId,shstGeometryId,shstReferenceId,geometry,hierarchy
24753,4075004794,4075001784,"['service', 'track']",service,0,1,1,753.540618,"[405292128, 405292129]",b6f6f29894314a39e1b7069bd4f53d96,4715d32ebbf18e592af26b06ce167ac0,"LINESTRING (-122.2364996 37.4191189, -122.2365...",12


In [600]:
from_list = non_unique_AB_links_df.shstReferenceId.tolist()
to_list = unique_AB_links_df.shstReferenceId.tolist()

drop_link_model_link_id_list = [c for c in from_list if c not in to_list]

In [601]:
link_MPO_gdf = link_MPO_gdf[~ link_MPO_gdf.shstReferenceId.isin(drop_link_model_link_id_list)]

In [602]:
shape_MPO_gdf = shape_MPO_gdf[shape_MPO_gdf.id.isin(link_MPO_gdf.id)].copy()

In [603]:
len(set(link_MPO_gdf.u.tolist() + link_MPO_gdf.v.tolist()))

643811

In [604]:
node_MPO_gdf.shape

(643811, 7)

In [605]:
link_MPO_gdf.id.nunique()

868567

In [606]:
shape_MPO_gdf.shape

(868567, 7)

In [607]:
link_MPO_gdf.shape

(1632702, 33)

In [608]:
link_MPO_gdf.groupby(["u", "v"]).shstReferenceId.count().shape

(1632702,)

In [609]:
node_MPO_gdf.osm_node_id.nunique()

643811

In [610]:
shape_MPO_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 868567 entries, 0 to 908266
Data columns (total 7 columns):
id                    868567 non-null object
fromIntersectionId    868567 non-null object
toIntersectionId      868567 non-null object
forwardReferenceId    868567 non-null object
backReferenceId       868567 non-null object
geometry              868567 non-null object
NAME                  868567 non-null object
dtypes: object(7)
memory usage: 53.0+ MB


In [611]:
link_MPO_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1632702 entries, 0 to 1666743
Data columns (total 33 columns):
access                1632702 non-null object
area                  1632702 non-null object
bike_access           1632702 non-null int64
bridge                1632702 non-null object
drive_access          1632702 non-null int64
est_width             1632702 non-null object
fromIntersectionId    1632702 non-null object
highway               1632702 non-null object
id                    1632702 non-null object
junction              1632702 non-null object
key                   1632702 non-null object
landuse               1632702 non-null object
lanes                 1632702 non-null object
link                  1632702 non-null object
maxspeed              1632702 non-null object
name                  1632702 non-null object
oneWay                1632702 non-null object
ref                   1632702 non-null object
roadway               1632702 non-null object
roundab

# Rename attributes

In [612]:
node_MPO_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 643811 entries, 0 to 643810
Data columns (total 7 columns):
osm_node_id     643811 non-null int64
shst_node_id    643811 non-null object
geometry        643811 non-null object
county          643811 non-null object
drive_access    643811 non-null int64
walk_access     643811 non-null int64
bike_access     643811 non-null int64
dtypes: int64(4), object(3)
memory usage: 39.3+ MB


In [613]:
shape_MPO_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 868567 entries, 0 to 908266
Data columns (total 7 columns):
id                    868567 non-null object
fromIntersectionId    868567 non-null object
toIntersectionId      868567 non-null object
forwardReferenceId    868567 non-null object
backReferenceId       868567 non-null object
geometry              868567 non-null object
NAME                  868567 non-null object
dtypes: object(7)
memory usage: 53.0+ MB


In [614]:
link_MPO_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1632702 entries, 0 to 1666743
Data columns (total 33 columns):
access                1632702 non-null object
area                  1632702 non-null object
bike_access           1632702 non-null int64
bridge                1632702 non-null object
drive_access          1632702 non-null int64
est_width             1632702 non-null object
fromIntersectionId    1632702 non-null object
highway               1632702 non-null object
id                    1632702 non-null object
junction              1632702 non-null object
key                   1632702 non-null object
landuse               1632702 non-null object
lanes                 1632702 non-null object
link                  1632702 non-null object
maxspeed              1632702 non-null object
name                  1632702 non-null object
oneWay                1632702 non-null object
ref                   1632702 non-null object
roadway               1632702 non-null object
roundab

In [615]:
not_to_export_link_json = ["id", "link"]

In [616]:
link_MPO_gdf.roadway.value_counts()

service           541982
residential       538666
footway           247489
cycleway           98615
tertiary           93947
secondary          68529
primary            26492
motorway_link       4983
trunk               4160
motorway            2774
secondary_link      1566
primary_link        1506
trunk_link          1382
tertiary_link        611
Name: roadway, dtype: int64

# Numbering Nodes

In [617]:
county_node_numbering_start_dict = {
    "San Francisco" : 1000000, 
    "San Mateo" : 1500000,
    "Santa Clara" : 2000000,
    "Alameda" : 2500000,
    "Contra Costa" : 3000000,
    "Solano" : 3500000,
    "Napa" : 4000000,
    "Sonoma" : 4500000,
    "Marin" : 5000000
}

In [618]:
node_MPO_gdf["model_node_id"] = node_MPO_gdf.groupby(["county"]).cumcount()

In [619]:
node_MPO_gdf["county_numbering_start"] = node_MPO_gdf["county"].map(county_node_numbering_start_dict)

node_MPO_gdf["model_node_id"] = node_MPO_gdf["model_node_id"] + node_MPO_gdf["county_numbering_start"]

In [620]:
node_MPO_gdf[node_MPO_gdf.model_node_id.isnull()].county.unique()

array([], dtype=object)

In [621]:
node_MPO_gdf.county.value_counts()

Santa Clara      192799
Alameda          125942
Contra Costa      97273
San Mateo         56363
Sonoma            56146
Solano            47318
San Francisco     27608
Marin             26529
Napa              13833
Name: county, dtype: int64

In [622]:
node_MPO_gdf.model_node_id.nunique()

643811

In [623]:
node_MPO_gdf[node_MPO_gdf.county.isin(county_node_numbering_start_dict.keys())].shape

(643811, 9)

# Numbering Links

In [624]:
county_link_numbering_start_dict = {
    "San Francisco" : 1, 
    "San Mateo" : 1000000,
    "Santa Clara" : 2000000,
    "Alameda" : 3000000,
    "Contra Costa" : 4000000,
    "Solano" : 5000000,
    "Napa" : 6000000,
    "Sonoma" : 7000000,
    "Marin" : 8000000
}

In [625]:
link_MPO_gdf.county.value_counts()

Santa Clara      516565
Alameda          315314
Contra Costa     237448
San Mateo        140927
Sonoma           135042
Solano           117102
San Francisco     73440
Marin             63184
Napa              33680
Name: county, dtype: int64

In [626]:
link_MPO_gdf["model_link_id"] = link_MPO_gdf.groupby(["county"]).cumcount()

link_MPO_gdf["county_numbering_start"] = link_MPO_gdf["county"].map(county_link_numbering_start_dict)

link_MPO_gdf["model_link_id"] = link_MPO_gdf["model_link_id"] + link_MPO_gdf["county_numbering_start"]

In [627]:
link_MPO_gdf[link_MPO_gdf.model_link_id.isnull()].county.unique()

array([], dtype=object)

In [628]:
link_MPO_gdf.model_link_id.nunique()

1632702

In [629]:
link_MPO_gdf[link_MPO_gdf.county.isin(county_link_numbering_start_dict.keys())].shape

(1632702, 35)

# Numbering Link A/B nodes

In [630]:
node_shst_model_id_dict = dict(zip(node_MPO_gdf.shst_node_id, node_MPO_gdf.model_node_id))

In [631]:
link_MPO_gdf["A"] = link_MPO_gdf["fromIntersectionId"].map(node_shst_model_id_dict)
link_MPO_gdf["B"] = link_MPO_gdf["toIntersectionId"].map(node_shst_model_id_dict)

In [632]:
link_MPO_gdf.head(3)

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,walk_access,wayId,width,county,geometry,length,model_link_id,county_numbering_start,A,B
0,nan,nan,1,nan,1,nan,505d64eb98f1da8d812a3b3801034308,service,208e093f10a62dcc6646a8efa0bde136,nan,...,1,231794292,nan,Contra Costa,"LINESTRING (-122.3315542 37.9812044, -122.3315...",84.645250,4000000,4000000,3000000,3007836
1,nan,nan,1,nan,1,nan,473979c78435732f01ca5a168afb62e0,residential,5fe3056a5583474c0c898983cd6a638b,nan,...,1,7864473,nan,Contra Costa,"LINESTRING (-121.94477 37.953322, -121.9443904...",509.283331,4000001,4000000,3000001,3077456
2,nan,nan,1,nan,1,nan,fc7b575d5d8c961d4a70fca846ae7f80,tertiary,38e962038ecf17c6c7394ba88bc3b4c1,nan,...,1,12183318,nan,Marin,"LINESTRING (-122.5398278 37.8979989, -122.5400...",128.106786,8000000,8000000,5000000,5014577


In [633]:
link_MPO_gdf[link_MPO_gdf.shstGeometryId == "38e962038ecf17c6c7394ba88bc3b4c1"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,walk_access,wayId,width,county,geometry,length,model_link_id,county_numbering_start,A,B
2,nan,nan,1,nan,1,nan,fc7b575d5d8c961d4a70fca846ae7f80,tertiary,38e962038ecf17c6c7394ba88bc3b4c1,nan,...,1,12183318,nan,Marin,"LINESTRING (-122.5398278 37.8979989, -122.5400...",128.106786,8000000,8000000,5000000,5014577
1413864,nan,nan,1,nan,1,nan,7550e87fc64657a10282672d814ab3c5,tertiary,38e962038ecf17c6c7394ba88bc3b4c1,nan,...,1,12183318,nan,Marin,"LINESTRING (-122.5398278 37.8979989, -122.5400...",128.106786,8053564,8000000,5014577,5000000


In [634]:
link_MPO_gdf[link_MPO_gdf.A.isnull()].county.value_counts()

Series([], Name: county, dtype: int64)

In [635]:
node_MPO_gdf[node_MPO_gdf.osm_node_id == 322507978]

,osm_node_id,shst_node_id,geometry,county,drive_access,walk_access,bike_access,model_node_id,county_numbering_start
643594,322507978,dddd914e805e1676c74d20ad2111ec59,POINT (-121.3149569 37.1691465),Santa Clara,0,1,1,2192698,2000000


In [636]:
node_MPO_gdf.model_node_id.nunique()

643811

# Write out

In [637]:
print("-------write out link shape geojson---------")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(shape_MPO_gdf, shape_prop)

with open("../../data/interim/step5_tidy_roadway/shape.geojson", "w") as f:
    json.dump(shape_geojson, f)

    
print("-------write out link json---------")

link_prop = link_MPO_gdf.drop(["geometry"] + not_to_export_link_json, axis = 1).columns.tolist()

out = link_MPO_gdf[link_prop].to_json(orient = "records")

with open('../../data/interim/step5_tidy_roadway/link.json', 'w') as f:
    f.write(out)

    
print("-------write out node geojson---------")

node_prop = node_MPO_gdf.drop("geometry", axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(node_MPO_gdf, node_prop)

with open("../../data/interim/step5_tidy_roadway/node.geojson", "w") as f:
    json.dump(node_geojson, f)  

-------write out link shape geojson---------
-------write out link json---------
-------write out node geojson---------


In [638]:
link_MPO_gdf.columns

Index(['access', 'area', 'bike_access', 'bridge', 'drive_access', 'est_width',
       'fromIntersectionId', 'highway', 'id', 'junction', 'key', 'landuse',
       'lanes', 'link', 'maxspeed', 'name', 'oneWay', 'ref', 'roadway',
       'roundabout', 'service', 'shstGeometryId', 'shstReferenceId',
       'toIntersectionId', 'tunnel', 'u', 'v', 'walk_access', 'wayId', 'width',
       'county', 'geometry', 'length', 'model_link_id',
       'county_numbering_start', 'A', 'B'],
      dtype='object')

In [639]:
print("-------write out link feather---------")

link_feather = link_MPO_gdf.reset_index(drop = True).drop("geometry", axis = 1).copy()

link_feather.to_feather(data_interim_dir + 'step5_tidy_roadway/link.feather')

-------write out link feather---------


In [640]:
link_feather.columns

Index(['access', 'area', 'bike_access', 'bridge', 'drive_access', 'est_width',
       'fromIntersectionId', 'highway', 'id', 'junction', 'key', 'landuse',
       'lanes', 'link', 'maxspeed', 'name', 'oneWay', 'ref', 'roadway',
       'roundabout', 'service', 'shstGeometryId', 'shstReferenceId',
       'toIntersectionId', 'tunnel', 'u', 'v', 'walk_access', 'wayId', 'width',
       'county', 'length', 'model_link_id', 'county_numbering_start', 'A',
       'B'],
      dtype='object')

In [641]:
link_df[link_df.u == 890045140][["roadway", "drive_access", "walk_access", "bike_access"]]

,roadway,drive_access,walk_access,bike_access
246811,service,1,1,1
903556,service,1,1,1
1598733,service,1,1,1


In [642]:
link_MPO_gdf[link_MPO_gdf.u == 890045140][["roadway", "drive_access", "walk_access", "bike_access"]]

,roadway,drive_access,walk_access,bike_access
241376,service,1,1,1
883586,service,0,1,1
1562251,service,1,1,1


In [643]:
link_df[link_df.shstReferenceId == "cb711b2cd73ebb6ac0548350ae9f226e"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,service,shstGeometryId,shstReferenceId,toIntersectionId,tunnel,u,v,walk_access,wayId,width
721952,no,nan,1,yes,1,nan,0be640da7eb02b44a6fbe14339f5b275,service,3b1b9350f00aa409e5e6ba1c508d6a98,nan,...,busway,3b1b9350f00aa409e5e6ba1c508d6a98,cb711b2cd73ebb6ac0548350ae9f226e,db1880b5dd0773e65d41989bf5775203,nan,5826305757,5826305760,1,616216455,nan


In [644]:
shape_gdf[shape_gdf.id == "3b1b9350f00aa409e5e6ba1c508d6a98"]

,id,fromIntersectionId,toIntersectionId,forwardReferenceId,backReferenceId,geometry
301211,3b1b9350f00aa409e5e6ba1c508d6a98,0be640da7eb02b44a6fbe14339f5b275,db1880b5dd0773e65d41989bf5775203,cb711b2cd73ebb6ac0548350ae9f226e,,"LINESTRING (-122.396039 37.7863407, -122.39615..."


In [645]:
link_MPO_gdf[link_MPO_gdf.shstReferenceId == "6233a20ff0d9dc29be597a06ffd19727"]

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id,junction,...,walk_access,wayId,width,county,geometry,length,model_link_id,county_numbering_start,A,B
